<a href="https://colab.research.google.com/github/guiomar/2021_workshop_accs_ohbm/blob/main/2021_workshop_OHBM_MEG_BIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MEG-BIDS conversion**

by *Guiomar Niso*


[ACCS and OHBM Workshop, 13th October, 2021]

##BIDS specification

**MEG-BIDS specification**: (Niso et al. 2018)

https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/02-magnetoencephalography.html

```
Niso G., Gorgolewski K.J., Bock E., Brooks T.L., Flandin G., Gramfort A., Henson R.N., Jas M., Litvak V., Moreau J., Oostenveld R., Schoffelen J.M., Tadel F., Wexler J., Baillet S. (2018). 
MEG-BIDS, the brain imaging data structure extended to magnetoencephalography.
Scientific data, 5(1), 1-5. doi:10.1038/sdata.2018.110
```

**EEG-BIDS specification**: (Pernet et al. 2019)

https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/03-electroencephalography.html

```
Pernet, C. R., Appelhoff, S., Gorgolewski, K.J., Flandin, G., Phillips, C., Delorme, A., Oostenveld, R. (2019). 
EEG-BIDS, an extension to the brain imaging data structure for electroencephalography. 
Scientific data, 6 (103). doi:10.1038/s41597-019-0104-8
```

##Toolboxes


**MNE-Python**
* Python
* MEG, EEG, iEEG
* **MNE-BIDS**: (Appelhoff et al., 2019): https://mne.tools/mne-bids/stable/



**Brainstorm**
* Matlab
* No coding needed (GUI)
* MEG, EEG, iEEG
* Tutorials for **exporting and importing** BIDS datasets: https://neuroimage.usc.edu/brainstorm/ExportBids

**Fieldtrip**
* Matlab
* MEG, EEG
* Funtion: **data2bids**: https://www.fieldtriptoolbox.org/reference/data2bids/


**EEGLAB**
* Matlab
* EEG
* **bids-matlab-tools** (Delorme et al. 2021): https://github.com/sccn/bids-matlab-tools

#BIDS conversion using MNE-BIDS

MEG-BIDS conversion using **MNE-BIDS** from MNE-Python

```
Appelhoff, S., Sanderson, M., Brooks, T., Vliet, M., Quentin, R., Holdgraf, C., Chaumon, M., Mikulan, E., Tavabi, K., Höchenberger, R., Welke, D., Brunner, C., Rockhill, A., Larson, E., Gramfort, A., & Jas, M. (2019). 
MNE-BIDS: Organizing electrophysiological data into the BIDS format and facilitating their analysis. 
Journal of Open Source Software, 4:1896. doi:10.21105/joss.01896
```



To use Google Drive:


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Import libraries:

In [38]:
import numpy as np
import pandas as pd
import os.path as op
import shutil

Install and import MNE-Python and MNE-BIDS
*   **MNE-Python**: https://mne.tools/stable
*   **MNE-BIDS**: https://mne.tools/mne-bids/stable/

In [ ]:
!pip install mne

In [ ]:
!pip install mne-bids

In [41]:
import mne
import mne_bids

Go to workshop folder:

In [ ]:
cd "/gdrive/MyDrive/Colab Notebooks/2021_Workshop_ACCS_OHBM/"

##Load data

In the Workshop we will be working with the **SPM multimodal dataset**.

MEG data could be downloaded from: https://www.fil.ion.ucl.ac.uk/spm/data/mmfaces

Read CTF *.ds folders with MNE

In [ ]:
fname = 'MEG/SPM_CTF_MEG_example_faces1_3D.ds'
raw = mne.io.read_raw_ctf(fname)

Fill in missing info:

In [43]:
raw.info['line_freq'] = 50  # Power line frequency

In [9]:
raw.info?

##Create BIDS path

In [44]:
out_path= './bids_dataset'
datatype = 'meg'
subject = '01'
task    = 'faces';
session = '01'
run     = None

bids_path = mne_bids.BIDSPath(subject=subject, session=session, run=run, 
                              task=task, datatype=datatype, root=out_path)
bids_path

BIDSPath(
root: bids_dataset
datatype: meg
basename: sub-01_ses-01_task-faces)

Remove folder if already exists:

In [ ]:
if op.exists(out_path):
    shutil.rmtree(out_path)
    print('Removed')

##Calibration files

Some MEG datasets from Elekta/Neuromag/MEGIN (fif) have fine-calibration and crosstalk files

*** NOT the case in this example** *



In [ ]:
cal_fname = 'SSS/sss_cal_mgh.dat'
ct_fname = 'SSS/ct_sparse_mgh.fif'

write_meg_calibration(cal_fname, bids_path)
write_meg_crosstalk(ct_fname, bids_path)

##Convert to BIDS

In [ ]:
mne_bids.write_raw_bids(raw, bids_path=bids_path) # events_data=events_data, event_id=event_id,

Print BIDS directory tree:

In [ ]:
mne_bids.print_dir_tree(out_path) #maxdepth

In [ ]:
mne_bids.print_dir_tree('./MEG')

Events:

In [ ]:
from mne_bids.stats import count_events
counts = count_events(out_path)
counts

## Mark bad channels

First, read BIDS dataset:

In [ ]:
raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False);

In [ ]:
print(f'Channels currently marked as bad:\n'
      f'    {", ".join(raw.info["bads"])}\n')

In [55]:
#bads = ['MLC11-2908', 'MLC12-2908']
bads = [] # mark all channels as good (overwrite=True)
mne_bids.mark_bad_channels(ch_names=bads, bids_path=bids_path, overwrite=True, verbose=False)

Resetting status and description for all channels.


##Additional metadata

Sidecar *_meg.json file

In [58]:
out_path= './bids_dataset'
datatype = 'meg'
subject = '01'
task    = 'faces';
session = '01'
run     = None
suffix = 'meg'

bids_path = mne_bids.BIDSPath(subject=subject, session=session, run=run, 
                              task=task, datatype=datatype,suffix=suffix, root=out_path)

In [ ]:
sidecar_path = bids_path.copy().update(extension='.json')
sidecar_path

In [ ]:
print(bids_path.match())

Define a dictionary with new entries

*** BE STRICT IN THE NAMING** *

In [60]:
entries = {
    'PowerLineFrequency': 60,
    'Manufacturer': "MEGIN"
}

In [ ]:
mne_bids.update_sidecar_json(bids_path=sidecar_path, entries=entries)

Add or rewrite a few more fields:

In [ ]:
entries['Manufacturer'] = "CTF"
entries['PowerLineFrequency'] = 50
mne_bids.update_sidecar_json(bids_path=sidecar_path, entries=entries)

# BIDS conversion using Brainstorm

Brainstorm toolbox (Tadel et al. 2011): https://neuroimage.usc.edu/brainstorm


Tutorials for **exporting and importing** BIDS datasets: https://neuroimage.usc.edu/brainstorm/ExportBids


Examples using BIDS datasets:

(Tadel et al. 2019)

```
Tadel F, Bock E, Niso G, Mosher JC, Cousineau M, Pantazis D, Leahy RM, Baillet S. (2019)
MEG/EEG Group Analysis With Brainstorm
Frontiers in Neuroscience, 13, 76. doi:10.3389/fnins.2019.00076
```

(Niso et al. 2019)
```
Niso G, Tadel F, Bock E, Cousineau M, Santos A, Baillet S. (2019)
Brainstorm Pipeline Analysis of Resting-State Data from the Open MEG Archive
Frontiers in Neuroscience, 13, 284. doi:10.3389/fnins.2019.00284
```

#BIDS Validator

Online BIDS validator: https://bids-standard.github.io/bids-validator/